<a href="https://colab.research.google.com/github/rehmanaly0051/CGAN-Model-Training/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np

In [19]:
IMAGE_SIZE = 128
BATCH_SIZE = 32
LATENT_DIM = 100
EPOCHS = 100
LEARNING_RATE = 0.0002

# ✅ Dataset
class FloorPlanDataset(Dataset):
       def __init__(self, csv_path, image_dir, transform=None):
           self.data = pd.read_csv(csv_path)
           self.image_dir = image_dir
           self.transform = transform

       def __len__(self):
           return len(self.data)

       def __getitem__(self, idx):
           row = self.data.iloc[idx]
           img_path = os.path.join(self.image_dir, os.path.basename(row['Image Path']))
           print("Image Path:", img_path)  # Print the image path
           image = Image.open(img_path).convert("RGB")
           if self.transform:
               image = self.transform(image)

           # Normalize attributes (keep consistent with training range)
           sqft = float(row['Square Feet']) / 5000.0
           beds = float(row['Beds']) / 6.0
           baths = float(row['Baths']) / 5.0
           garages = float(row['Garages']) / 3.0

           condition = torch.tensor([sqft, beds, baths, garages], dtype=torch.float32)
           return image, condition

In [20]:
class Generator(nn.Module):
    def __init__(self, condition_dim=4, latent_dim=100):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, IMAGE_SIZE * IMAGE_SIZE * 3),
            nn.Tanh()
        )

    def forward(self, noise, condition):
        x = torch.cat([noise, condition], dim=1)
        img = self.model(x)
        return img.view(-1, 3, IMAGE_SIZE, IMAGE_SIZE)

In [21]:
class Discriminator(nn.Module):
    def __init__(self, condition_dim=4):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(IMAGE_SIZE * IMAGE_SIZE * 3 + condition_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, condition):
        x = torch.cat([img.view(img.size(0), -1), condition], dim=1)
        validity = self.model(x)
        return validity

In [22]:
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = FloorPlanDataset("/content/drive/MyDrive/archive/house_plans_details.csv", "/content/drive/MyDrive/archive/images", transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [23]:
import os
print(os.getcwd())

/content


In [24]:
generator = Generator()
discriminator = Discriminator()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=49156, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [25]:
adversarial_loss = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LEARNING_RATE)

In [26]:
for epoch in range(EPOCHS):
    for i, (imgs, conditions) in enumerate(dataloader):
        real_imgs = imgs.to(device)
        conditions = conditions.to(device)
        batch_size = real_imgs.size(0)

        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        # ------------------ Train Generator ------------------
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, LATENT_DIM, device=device)
        gen_imgs = generator(z, conditions)
        g_loss = adversarial_loss(discriminator(gen_imgs, conditions), valid)
        g_loss.backward()
        optimizer_G.step()

        # ------------------ Train Discriminator ------------------
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs, conditions), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach(), conditions), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

    print(f"Epoch {epoch+1}/{EPOCHS} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    if (epoch + 1) % 10 == 0:
        save_image(gen_imgs.data[:8], f"generated_{epoch+1}.png", normalize=True)

Image Path: /content/drive/MyDrive/archive/images/73_1.jpg
Image Path: /content/drive/MyDrive/archive/images/9_22.jpg


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/archive/images/9_22.jpg'